# Investigating AC TF Input files
This notebook investigates the TF Input files that were created during the sprint at MPIK on 24/03/2020. There seems to be a problem with the scaling during the transformation from TF Input file to TF file. I will begin by checking the TF input files against the old TF Input file for the module

In [31]:
import fitsio
from target_calib import TFInputArrayReader
import numpy as np
from matplotlib import pyplot as plt
%matplotlib widget

In [2]:
paths = {
    "old": "/Users/Jason/Downloads/tempdata/CHEC-S_tf_data/SN0067/TFInput_File_SN0067_180213.tcal",
    "T20": "/Users/Jason/Downloads/tempdata/mpik_tf/tfinputs/TFInput_File_SN0067_20.tcal",
    "T25": "/Users/Jason/Downloads/tempdata/mpik_tf/tfinputs/TFInput_File_SN0067_25.tcal",
    "T30": "/Users/Jason/Downloads/tempdata/mpik_tf/tfinputs/TFInput_File_SN0067_30.tcal",
    "T35": "/Users/Jason/Downloads/tempdata/mpik_tf/tfinputs/TFInput_File_SN0067_35.tcal",
    "T40": "/Users/Jason/Downloads/tempdata/mpik_tf/tfinputs/TFInput_File_SN0067_40.tcal",
    "T45": "/Users/Jason/Downloads/tempdata/mpik_tf/tfinputs/TFInput_File_SN0067_45.tcal"
}

In [14]:
def read_tf_fitsio(path):
    with fitsio.FITS(path) as file:
        header = file[0].read_header()
        n_pixels = int(header['TM'] * header['PIX'])
        n_cells = int(header['CELLS'])
        n_amplitudes = int(header['PNTS'])
        data = file["DATA"].read(columns="CELLS").reshape((n_pixels, n_cells, n_amplitudes))
        amplitudes = file["AMPLITUDES"].read(columns="CELLS").astype('float64')
        return data, amplitudes

In [24]:
fig, ax = plt.subplots()
for key, path in paths.items():
    tf, amplitudes = read_tf_fitsio(path)
    ax.plot(amplitudes, tf[0, 0], label=key)
ax.set_xlabel("Input Amplitude (V)")
ax.set_ylabel("Pedestal-subtracted ADC")
ax.legend(loc='best')

/Users/jason/opt/anaconda3/envs/cta/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

The input amplitude appears to be incorrectly calibrated by a scale factor. The lab setup should be checked (e.g. load setting for the pulse generator).

## Check to see if TargetCalib can be used instead of fitsio

In [32]:
def read_tf_targetcalib(path):
    reader = TFInputArrayReader(paths['old'])
    tf = np.array(reader.GetTFInput())
    amplitudes = np.array(reader.GetAmplitudes())
    return tf, amplitudes

In [33]:
fig, ax = plt.subplots()
for key, path in paths.items():
    tf, amplitudes = read_tf_fitsio(path)
    ax.plot(amplitudes, tf[0, 0], label=key)
ax.set_xlabel("Input Amplitude (V)")
ax.set_ylabel("Pedestal-subtracted ADC")
ax.legend(loc='best')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …